# Installazione delle librerie
Visto che colab e' un ambiente di runtime provvisorio, eseguire questo script per ogni prima instanza della macchina, cosi da settare l'ambiente prima dell'esecuzione.

In [2]:
# .\.venv\Scripts\activate ---> for set local environment

! pip install pdfplumber
! pip install transformers
! pip install PyPDF2
! pip install spacy

! pip install torch
! pip install torchvision 
! pip install torchaudio

# ! pip install https://huggingface.co/bullmount/it_nerIta_trf/resolve/main/it_nerIta_trf-any-py3-none-any.whl
! python -m spacy download it_core_news_lg
! python -m spacy download it_core_news_sm


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached https://github.com/explosion/spacy-models/releases/download/it_core_news_lg-3.7.0/it_core_news_lg-3.7.0-py3-none-any.whl (567.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------- 0.0/13.0 MB 495.5 kB/s eta 0:00:27
     --------------------------------------- 0.1/13.0 MB 819.2 kB/s eta 0:00:16
      --------------------------------------- 0.3/13.0 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.3/13.0 MB 1.8 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/13.0 MB 2.9 MB/s eta 0:00:05
     ---- ----------------------------------- 1.3/13.0 MB 4.7 MB/s eta 0:00:03
     ---- ----------------------------------- 1.3/13.0 MB 4.7 MB/s eta 0:00:03
     ----- ---------------------------------- 1.7/13.0 MB 4.4 MB/s eta 0:00:03
     ---------- ----------------------------- 3.3/13.0 MB 7.6 MB/s eta 0:00:02
     ------------ --------------------------- 4.1/13.0 MB 8.2 MB/s eta 0:00:02
     ---------------- ----------------------- 5.3/13.0 MB 9.8 MB


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip freeze >> requirements.txt

In [12]:
# spiegazione di come funziona https://www.youtube.com/watch?v=fgmA9am9qCo&ab_channel=ASLearning

import spacy
from PyPDF2 import PdfReader
from transformers import BartTokenizer, BartForConditionalGeneration
from spacy import displacy


percorso_file_pdf = './dispense-neuroetica-ed-etica-applicata-mfil03-1-.pdf'
# percorso_file_pdf = 'digitalinnovation.pdf'

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Loading spacy models
nlp = spacy.load("it_core_news_lg")
# nlpNer = spacy.load("it_nerIta_trf")

In [ ]:
#Used model it_core_news_lg
#SPLIT INFO PARAGRAPH

# Read PDF Files
with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()
doc = nlp(text)

paragrafi = []
paragrafo_corrente = ""

for sent in doc.sents:
    paragrafo_corrente += sent.text

    if len(paragrafo_corrente) >= 200:
        paragrafi.append(paragrafo_corrente)
        paragrafo_corrente = ""

if paragrafo_corrente:
    paragrafi.append(paragrafo_corrente)


In [ ]:
#Used model it_nerIta_trf

#NER for page MODEL1
with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    num = 0
    # Estrai il testo da ciascuna pagina del PDF
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        print('--------------------------------------PAGE-----------------------------------------', num)
        pageText = page.extract_text()
        doc = nlpNer(pageText)
        displacy.render(doc, style="ent",jupyter=True)
        num = num + 1

In [ ]:
#Used model it_nerIta_trf

#NER for page MODEL1
with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    num = 0
    # Estrai il testo da ciascuna pagina del PDF
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        print('--------------------------------------PAGE-----------------------------------------', num)
        pageText = page.extract_text()
        doc = nlp(pageText)
        displacy.render(doc, style="ent",jupyter=True)
        num = num + 1

In [ ]:
doc = nlp('il pesce è ottimo')
print('Entità:', doc.ents)



Entità: ()


In [14]:
#SUMMARIZATION for paragraph
#For finetuning https://towardsdatascience.com/fine-tuning-the-bart-large-model-for-text-summarization-3c69e4c04582

for paragraph in paragrafi:
  inputs = tokenizer([paragraph], return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'], max_length=100, early_stopping=False)
  print([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])

NameError: name 'paragrafi' is not defined

In [33]:
#SUMMARIZATION for page
token_limit_model = 1030

with open(percorso_file_pdf, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    num = 1
    # Estrai il testo da ciascuna pagina del PDF
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        print('--------------------------------------PAGE-----------------------------------------', num)
        num = num + 1
        pageText = page.extract_text()
        inputs = tokenizer([pageText], return_tensors='pt')
        token_inside_page = len(inputs['input_ids'][0])
        print("token_inside_page", token_inside_page)

        if token_inside_page >= token_limit_model:
            continue

        summary_ids = model.generate(inputs['input_ids'], max_length=100, early_stopping=False)
        print([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])


--------------------------------------PAGE----------------------------------------- 1
token_inside_page 122
['Corso di Laurea Magistrale in Scienze Filosofiche Applicate (LM -78) is based in Venice, Italy. Insegnamento:   \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0\xa0Neuroetica ed etica applicata (M -FIL/03) is conducted by Prof. Mirko Di Bernardo.']
--------------------------------------PAGE----------------------------------------- 2
token_inside_page 780
['1.1 Premes sa scopo del corso ........................................................................  3.2 Questioni terminologiche e definitorie ..............................  4.3 La struttura teoretica dell’etica normativa..................................................  6.4 La tripartizione storica etico -normativa: etiche teleologiche, deontologiche.']
--------------------------------------PAGE----------------------------------------- 3
token_inside_page 163
['5.1 Lo Human Enhancement  .........................................

IndexError: index out of range in self

In [ ]:
#